Librosa - example for reading audio files - .wav 

In [0]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276
#3) https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?Expires=1585601065&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=Lnpf6wT8rkozSh9av7U9nGuC7WAH6KuI2Cj3Y7G366gkGlh8D-Ie1Kc~TyBAUu~uMsVltleJcSv3p6TCm6HdFnhpyoTgLcYh6eFfvQwIUqbk1Bf4JZldgB~BDKUOwY1G0pA-HoKjvIAu3avO98SMO35upakm9OEBByd4nC9aXsjKRThd6bTpq1qIuuD9gh1l5FaM6hNRB0c2lCf4Q3adx7C3FW0NMwdWhcuF45A9f~dO3zTWWSQamoo5Otc-PHMMt96TetNcML~jy9ghgJeCPY6DJLUIwQAt03fENBluS~TjTJ17WD~n51xiRofb94fEJHoRHh0d-430LLwr7BX4IA__

In [1]:
!pip install pydub

In [2]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=73df421222b8b0e6286e6f1f0c9b44cdd305eb2f91d07b7f02ff6afe61aff7b8
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [3]:
!pip install pysptk

     |████████████████████████████████| 419kB 8.4MB/s 
  Created wheel for pysptk: filename=pysptk-0.1.18-cp36-cp36m-linux_x86_64.whl size=949957 sha256=b0f1aba1a668643f4d6dfd7a351d24dd61164a664111e27d999d911bdb12eb74
  Stored in directory: /root/.cache/pip/wheels/c7/96/d2/a163240019c59504402fab713af259026af81a99dea943404a
Successfully built pysptk


In [0]:
import numpy as np
import os
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
from pydub.playback import play
import matplotlib.pyplot as plt
#import seaborn as sn
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import kurtosis, skew
from scipy.signal import lfilter
import librosa
import pysptk
import math
import sys
import random
random.seed(1)
#settings
import config


## Reading recordings

In [0]:
all_s=[]
all_label=[]
all_id=[]
all_fs=[]

In [3]:
#Read wav data set

if config.featExtr_skip is False:
    print("Readings wavs...")

    #only list files in FOLDER_PATH directory
    wav_files = [f for f in os.listdir(config.FOLDER_PATH) if os.path.isfile(os.path.join(config.FOLDER_PATH, f))]
    for file_name in wav_files:
    
        fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
        #full path file name
        full_fname = config.FOLDER_PATH+file_name
        print(full_fname)
    
        # load audio
        s = AudioSegment.from_wav(full_fname)
        print(full_fname)
        all_s.append(s)
        #sampling rate:
        info = mediainfo(full_fname)
        fs = float(info['sample_rate'])
        all_fs.append(fs)
    
        #get ID of recording
        ID = fname_noExt.split('-')[-2] #for the current type of naming
        #print(file_name)
        #print(ID)
        all_id.append(ID)
        words = ['covid', 'suspect', 'healthy']
        #for l in words:
         #   if l in fname_noExt:
          #      label = l
        #get label
        label = fname_noExt.split('-')[-1] #for the current type of naming #Change this for updated data 
        #print(label)
        all_label.append(label)

Readings wavs...
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 46-dg-I9j76-t8-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 46-dg-I9j76-t8-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 14 - After work-1UDFq2InljM-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 14 - After work-1UDFq2InljM-Dry.wav
data/YT_set/edited_wavs/edit_# 34 coughing up crap again-rkF_uMizqoc-Wet.wav
data/YT_set/edited_wavs/edit_# 34 coughing up crap again-rkF_uMizqoc-Wet.wav
data/YT_set/edited_wavs/edit_# 61 morning phlegmy cough...again-qfpJg179YNk-Wet.wav
data/YT_set/edited_wavs/edit_# 61 morning phlegmy cough...again-qfpJg179YNk-Wet.wav
data/YT_set/edited_wavs/edit_Dry Afternoon Cough-6LK6yHtIung-Dry.wav
data/YT_set/edited_wavs/edit_Dry Afternoon Cough-6LK6yHtIung-Dry.wav
data/YT_set/edited_wavs/edit_November cold (wet coughing)-DYfjPnty2Ho-Wet.wav
data/YT_set/edi

In [5]:
len(wav_files) # Total 36 files uploaded 

36

Listening to some of the audios

In [0]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Dry')

In [0]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Wet')

In [0]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='suspect')

In [0]:
if config.featExtr_skip is False:
    s=all_s[15]
    s

## Feature extraction

In [8]:
import numpy as np
np.shape(all_label)


(36,)

In [0]:
all_label #Fix this - should contain - healthy , covid, suspect - eventually classify suspect VS covid

In [7]:
import featureExtractionFunctions as featExtrLib
import pysptk

if config.featExtr_skip is False:

    feats = featExtrLib.feature_extraction_Step(all_s,all_id,all_label)
    
       
#Lenght of all_s, all_id and all_label must be the same
       #Lenght is 36, of a ll arguments 

High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass filtering...
Computing features...
High-pass 

In [8]:
feats.shape

(4470, 25)

## Load  (or store) features 

In [0]:
feats_fname = 'feats_df.pkl'

if config.featExtr_skip is False:
    #Store feature df
    feats.to_pickle(feats_fname)
else:
    #Load feature df
    feats = pd.read_pickle(feats_fname)

In [12]:

pandas2ri.activate()

base = importr('base')
# call an R function on a Pandas DataFrame
base.summary(feats_fname)

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/vectors.py:927: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '


array(['1', 'character', 'character'], dtype='<U9')

## Pre-processing of features

In [0]:
feats2 = featExtrLib.processingNaNvalues(feats)

In [0]:
label_dict = featExtrLib.createLabelDict_addLabel2df(feats2)
mean_std_feats = featExtrLib.frame_mean_std_chunk_modeling (feats2,label_dict)

In [11]:
mean_std_feats.shape

(465, 101)

## Model training

In [0]:
X_train = mean_std_feats.drop(['label','Id','subIdx'], 1).copy()
y_train =  mean_std_feats['label'].copy()

ID_train = mean_std_feats['Id']


In [12]:
y_train #465

0      Dry
1      Dry
2      Dry
3      Dry
4      Dry
      ... 
460    Dry
461    Dry
462    Dry
463    Dry
464    Dry
Name: label, Length: 465, dtype: object

In [13]:
X_train.shape

(465, 98)

In [0]:
import modelTrainingFunctions as modelTrainLib

pred_probs = modelTrainLib.modelTraining(X_train,y_train,ID_train)

In [0]:
mean_pred_probs = modelTrainLib.get_predClass_per_audio(pred_probs, label_dict)


#Change here 

## Evaluation

In [14]:
#Change script to multiclass labeling

import classifEvaluationFunctions as evalLib
evalLib.evaluation_Step(mean_pred_probs)
    


#Scores:
#Accuracy: 0.583333
#Precision: 0.583333
#F1-score: 0.580420
#Recall: 0.585714

#Confusion matrix
#pred_class  Dry  Wet  All
#label                    
#Dry           9    6   15
#Wet           9   12   21
#All          18   18   36

Scores:
Accuracy: 0.750000
Precision: 0.750000
F1-score: 0.748252
Recall: 0.757143

Confusion matrix
pred_class  Dry  Wet  All
label                    
Dry          12    3   15
Wet           6   15   21
All          18   18   36


In [0]:
#Export the prediction model for SGD

#After training export the trained model using pickle REf :https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
import pickle
filename = "/home/sap007/Master-Thesis/Streamlit Demo_LA/model_Dec2vec.sav"
pickle.dump(model,open(filename,'wb'))
print("File exported")

In [0]:
y_train = y_train.astype('category')
y_train = y_train.cat.codes
y_train.head(10)

In [0]:
import modelTrainingFunctions_CNN as modelTrainLib

pred_probs = modelTrainLib.modelTraining(X_train,y_train,ID_train)

In [0]:
y.values()

In [0]:
plt.title('Accuracy')
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.legend()
plt.show();

In [0]:
# Evaluating the model on the training and testing set
score = model.evaluate(xx_train, yy_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(xx_test, yy_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))


#Training Accuracy: 95.97%
#Testing Accuracy: 76.34%


#Training Accuracy: 98.77%
#Testing Accuracy: 72.14%


#Training Accuracy: 92.58%
#Testing Accuracy: 85.11%

#Training Accuracy: 84.45%
#Testing Accuracy: 82.98%

#Training Accuracy: 82.54%
#Testing Accuracy: 74.47%

#Training Accuracy: 83.73%
#Testing Accuracy: 80.85%


#Training Accuracy: 84.45%
#Testing Accuracy: 82.98%

#Training Accuracy: 84.21%
#Testing Accuracy: 82.98%


Training Accuracy: 79.94%
Testing Accuracy: 77.78%


Fine tune the parameters to improve classification accuracy - if not then refine the features used - perhaps add weightage somewhere - aim for atleast 80% accuracy 

Generation 15 - Current best internal CV score: 0.7109589041095891

Best pipeline: DecisionTreeClassifier(PolynomialFeatures(CombineDFs(input_matrix, input_matrix), degree=2, include_bias=False, interaction_only=False), criterion=entropy, max_depth=3, min_samples_leaf=18, min_samples_split=16)

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=15,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=50,
               random_state=42, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

Sequence Modeling 


hi, i just wanted to mention, about the modeling of features using sequence models. the current super basic way that I used is modeling chunks of X consecutive frames (so far I used 10frames) computing their mean and standard deviation.
3:05
You can use maybe this function as model. It is within the featureExtractionFunctions.py script




